# RISER Pipeline

End-to-end pipeline combining:
1. **Generation** — multi-model stakeholder elicitation via live APIs
2. **Evaluation** — salience scoring with DeepSeek (Mitchell's model)
3. **Clustering** — embedding-based grouping with adaptive k-selection
4. **Visualization** — interactive cluster explorer

> Run sections sequentially. Each section saves its output CSV for the next.

## Section 1 — Setup

In [1]:
# ── Imports ─────────────────────────────────────────────────────────────────
import json
import time
import requests
import numpy as np
import pandas as pd
import torch
from collections import defaultdict, deque
from typing import Dict, List
from scipy.stats import entropy
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from sklearn.cluster import KMeans
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from IPython.display import display, HTML


## Section 2 — Configuration

Set your project description and API keys here.

In [ ]:
# ── Project ──────────────────────────────────────────────────────────────────
PROJECT_NAME = "AI-powered student assessment"
PROJECT_DESCRIPTION = """
    An AI-powered student assessment system that uses machine learning to evaluate
    student performance, predict learning outcomes, and provide personalized recommendations
    for educational interventions. The system analyzes student data including test scores,
    attendance records, and behavioral patterns to identify at-risk students.
"""

# ── API Keys ──────────────────────────────────────────────────────────────────
API_KEYS = {
    "deepinfra": "",
    "openai":    "",
    "anthropic": "",
    "google":    ""
}

# ── Generation parameters ─────────────────────────────────────────────────────
MAX_REPEATS_PER_MODEL = 3    # Max times to call each model/temp combo
COST_CAP              = 5.0  # USD
ITEM_CAP              = 200  # Max accepted stakeholders
YIELD_THRESHOLD       = 3    # Min new items per call before saturation
SIM_THRESHOLD         = 0.8  # Cosine similarity threshold for deduplication

# ── Evaluation parameters ─────────────────────────────────────────────────────
DEEPINFRA_API_KEY  = API_KEYS['deepinfra']
DEEPINFRA_BASE_URL = 'https://api.deepinfra.com/v1/openai/chat/completions'
DEEPSEEK_MODEL     = 'deepseek-ai/DeepSeek-V3.1-Terminus'

# ── Clustering parameters ─────────────────────────────────────────────────────
EMBEDDING_MODEL_NAME = 'all-MiniLM-L6-v2'          # Used by generation
EMBEDDING_MODEL      = 'sentence-transformers/all-MiniLM-L6-v2'  # Used by clustering
BATCH_SIZE           = 64
SCORE_WEIGHT         = 0.3   # Weight for salience scores vs embeddings
EMBEDDING_OUTPUT     = 'stakeholder_embeddings.npy'
K_MIN, K_MAX, K_STEP = 10, 50, 1
DIST_EPS, VAR_EPS    = 1e-6, 1e-6
PATIENCE             = 1

# ── RHRC configuration ────────────────────────────────────────────────────────
MIN_CHILD_SIZE  = 3   # ST1: refuse split if any child would have fewer points
MIN_PARENT_SIZE = 6   # ST2: refuse split if parent has fewer points

# ── Output files ─────────────────────────────────────────────────────────────
GENERATION_OUTPUT    = 'live_api_results_detailed.csv'
FREQ_TABLE_OUTPUT    = 'frequency_table_with_rejected.csv'
EVALUATION_OUTPUT    = 'stakeholder_salience_evaluated.csv'
CLUSTERED_OUTPUT     = 'stakeholder_clustered.csv'

print('✓ Configuration loaded')
print(f'  Project : {PROJECT_NAME}')
print(f'  Cost cap: ${COST_CAP}  |  Item cap: {ITEM_CAP}')


✓ Configuration loaded
  Project : AI-powered student assessment
  Cost cap: $5.0  |  Item cap: 200


## Section 3 — Stakeholder Generation

Queries multiple LLMs via live APIs to build a deduplicated stakeholder knowledge base.
Outputs `live_api_results_detailed.csv` and `frequency_table_with_rejected.csv`.

In [3]:
# ── Load embedding model (used for deduplication during generation) ───────────
print(f'Loading embedding model ({EMBEDDING_MODEL_NAME})...')
embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME)
print('✓ Model loaded')


Loading embedding model (all-MiniLM-L6-v2)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✓ Model loaded


In [4]:
# ── Generation prompts ────────────────────────────────────────────────────────
GENERATION_SYSTEM_PROMPT = """You are an expert in ethics requirements engineering.
Your task is to identify and justify which stakeholders and vulnerable groups
should be interviewed to uncover ethical issues for a given AI or software system.
Return your final answer in the specified JSON format only."""

JSON_SCHEMA = """{
  "stakeholders": [{"role": "", "justification": ""}],
  "vulnerable_groups": [{"group": "", "justification": ""}],
  "reflection_summary": ""
}"""

USER_PROMPT_TEMPLATE = """
PROJECT DESCRIPTION:
<<<
{project_description}
>>>

TASK INSTRUCTIONS:

[CREATIVE PROMPT SECTION]
Imagine you are an ethics-aware requirements engineer preparing stakeholder interviews.
Given the project description above, creatively—but realistically—propose the key
stakeholders and specific vulnerable end-users to interview to elicit ethical requirements.
Use the questions below as a guide.

QUESTIONS:
1- To whom could the AI system have legal obligations?
2- Who might be [positively or negatively] impacted by the AI system's decisions or activities?
3- Who is likely to express concerns about the decisions and activities of the AI system?
4- Who has been involved in the past when concerns about AI systems needed to be addressed?
5- Who can help the AI system address specific impacts and meet its responsibilities?
6- Who would be disadvantaged if excluded from the engagement with the AI system?

[ATOMICITY RULES - STRICT]
- Each entry must contain ONE atomic actor type only.
- No Role Conflation (e.g., NO "teachers and administrators"), INSTEAD separate entries.
- No umbrella categories (e.g., NO "regulatory bodies (e.g., education, health)"), INSTEAD separate entries.
- If a role could be separated into two distinct units, separate them.

[CANONICALIZATION RULES]
- Use the most standard, domain-recognizable title.
- Avoid synonyms, paraphrases, stylistic variations, or repeated variants.
- Merge duplicated roles under a single canonical label.

[REALISM FILTER]
- Only include roles that commonly exist in real-world systems.
- Do not invent new committees, institutions, technical roles, or regulatory bodies.

[REFLECTION PROMPT SECTION]
Reflect on your initial list:
1. Did you omit any category of stakeholders?
2. Did you consider indirect or silent stakeholders?
3. Are any of your listed groups unrealistic or redundant?

Now revise your final concise list of:
- Stakeholders to interview (with 1-2 sentence justification each)
- Vulnerable end-users or groups to include (with 1-2 sentence justification each)

OUTPUT FORMAT:
Return your final output in JSON exactly as follows:
{json_schema}
"""


In [5]:
# ── Model registry & cluster config ──────────────────────────────────────────
MODEL_TO_PROVIDER = {
    "gpt-5-mini-2025-08-07":                              "openai",
    "gpt-5.1-2025-11-13":                                 "openai",
    "claude-haiku-4-5":                                   "anthropic",
    "claude-sonnet-4-5":                                  "anthropic",
    "gemini-2.5-flash":                                   "google",
    "gemini-3-pro-preview":                               "google",
    "openai/gpt-oss-20b":                                 "deepinfra",
    "openai/gpt-oss-120b":                                "deepinfra",
    "Qwen/Qwen3-30B-A3B":                                 "deepinfra",
    "mistralai/Mixtral-8x7B-Instruct-v0.1":               "deepinfra",
    "microsoft/WizardLM-2-8x22B":                         "deepinfra",
    "microsoft/phi-4":                                    "deepinfra",
    "meta-llama/Llama-4-Scout-17B-16E-Instruct":          "deepinfra",
    "deepseek-ai/DeepSeek-V3.1-Terminus":                 "deepinfra",
    "google/gemma-3-27b-it":                              "deepinfra",
    "meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8":  "deepinfra",
}

MODEL_REGISTRY = {
    "openai/gpt-oss-20b":    {"tier": 0, "cost": 0.19, "variations": {0.3: {"score": 0.85, "pure_score": 0.79}, 0.7: {"score": 0.83, "pure_score": 0.76}, 0.0: {"score": 0.81, "pure_score": 0.71}}},
    "openai/gpt-oss-120b":   {"tier": 1, "cost": 0.30, "variations": {0.7: {"score": 0.77, "pure_score": 0.66}, 0.3: {"score": 0.76, "pure_score": 0.65}, 0.0: {"score": 0.74, "pure_score": 0.62}}},
    "gpt-5-mini-2025-08-07": {"tier": 0, "cost": 2.00, "variations": {0.0: {"score": 0.83, "pure_score": 0.81}}},
    "gpt-5.1-2025-11-13":    {"tier": 1, "cost": 10.00,"variations": {0.0: {"score": 0.67, "pure_score": 0.83}}},
    "Qwen/Qwen3-30B-A3B":                                {"tier": 0, "cost": 0.28, "variations": {0.7: {"score": 0.64, "pure_score": 0.47}, 0.3: {"score": 0.62, "pure_score": 0.44}, 0.0: {"score": 0.50, "pure_score": 0.26}}},
    "mistralai/Mixtral-8x7B-Instruct-v0.1":             {"tier": 1, "cost": 0.54, "variations": {0.7: {"score": 0.62, "pure_score": 0.45}, 0.3: {"score": 0.58, "pure_score": 0.39}, 0.0: {"score": 0.58, "pure_score": 0.38}}},
    "microsoft/WizardLM-2-8x22B":                       {"tier": 2, "cost": 0.48, "variations": {0.7: {"score": 0.63, "pure_score": 0.45}, 0.3: {"score": 0.60, "pure_score": 0.41}, 0.0: {"score": 0.57, "pure_score": 0.38}}},
    "microsoft/phi-4":                                  {"tier": 3, "cost": 0.14, "variations": {0.7: {"score": 0.66, "pure_score": 0.49}, 0.3: {"score": 0.58, "pure_score": 0.38}, 0.0: {"score": 0.45, "pure_score": 0.18}}},
    "meta-llama/Llama-4-Scout-17B-16E-Instruct":        {"tier": 4, "cost": 0.30, "variations": {0.7: {"score": 0.60, "pure_score": 0.40}, 0.3: {"score": 0.59, "pure_score": 0.40}, 0.0: {"score": 0.45, "pure_score": 0.18}}},
    "deepseek-ai/DeepSeek-V3.1-Terminus":               {"tier": 0, "cost": 0.79, "variations": {0.3: {"score": 0.64, "pure_score": 0.48}, 0.7: {"score": 0.63, "pure_score": 0.48}, 0.0: {"score": 0.62, "pure_score": 0.45}}},
    "gemini-2.5-flash":                                 {"tier": 1, "cost": 2.50, "variations": {0.7: {"score": 0.63, "pure_score": 0.53}, 0.3: {"score": 0.63, "pure_score": 0.53}, 0.0: {"score": 0.52, "pure_score": 0.37}}},
    "claude-sonnet-4-5":                                {"tier": 2, "cost": 15.00,"variations": {0.7: {"score": 0.43, "pure_score": 0.65}, 0.3: {"score": 0.40, "pure_score": 0.60}, 0.0: {"score": 0.30, "pure_score": 0.44}}},
    "google/gemma-3-27b-it":                            {"tier": 3, "cost": 0.16, "variations": {0.7: {"score": 0.64, "pure_score": 0.47}, 0.3: {"score": 0.61, "pure_score": 0.42}, 0.0: {"score": 0.59, "pure_score": 0.40}}},
    "claude-haiku-4-5":                                 {"tier": 3, "cost": 5.00, "variations": {0.7: {"score": 0.63, "pure_score": 0.62}, 0.3: {"score": 0.61, "pure_score": 0.59}, 0.0: {"score": 0.42, "pure_score": 0.29}}},
    "gemini-3-pro-preview":                             {"tier": 4, "cost": 12.00,"variations": {0.7: {"score": 0.36, "pure_score": 0.44}, 0.3: {"score": 0.35, "pure_score": 0.43}, 0.0: {"score": 0.16, "pure_score": 0.14}}},
    "meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8":{"tier": 4, "cost": 0.60, "variations": {0.7: {"score": 0.56, "pure_score": 0.35}, 0.3: {"score": 0.52, "pure_score": 0.30}, 0.0: {"score": 0.54, "pure_score": 0.33}}},
}

def build_cluster_config():
    return {
        4: ["openai/gpt-oss-120b", "claude-haiku-4-5", "claude-sonnet-4-5"],
        3: ["openai/gpt-oss-20b", "gpt-5-mini-2025-08-07", "gpt-5.1-2025-11-13"],
        2: ["Qwen/Qwen3-30B-A3B", "mistralai/Mixtral-8x7B-Instruct-v0.1", "microsoft/WizardLM-2-8x22B", "microsoft/phi-4", "meta-llama/Llama-4-Scout-17B-16E-Instruct"],
        1: ["deepseek-ai/DeepSeek-V3.1-Terminus", "gemini-2.5-flash", "google/gemma-3-27b-it", "gemini-3-pro-preview", "meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8"]
    }


In [6]:
# ── LiveAPIProvider class ─────────────────────────────────────────────────────
class LiveAPIProvider:

    def __init__(self, api_keys, project_description, max_repeats):
        self.api_keys = api_keys
        self.project_description = project_description
        self.max_repeats = max_repeats
        self.call_counts = defaultdict(int)
        self.max_comp = 20000

    def call_api(self, model_name, temperature, max_retries=3):
        """Route API call to the appropriate provider"""
        provider = MODEL_TO_PROVIDER.get(model_name)
        if provider == "openai":    return self.call_openai(model_name, temperature, max_retries)
        elif provider == "anthropic": return self.call_anthropic(model_name, temperature, max_retries)
        elif provider == "google":  return self.call_google(model_name, temperature, max_retries)
        elif provider == "deepinfra": return self.call_deepinfra(model_name, temperature, max_retries)
        else: print(f"    ✗ Unknown provider for model: {model_name}"); return None, 0

    def _build_payload_messages(self):
        user_prompt = USER_PROMPT_TEMPLATE.format(
            project_description=self.project_description, json_schema=JSON_SCHEMA)
        return [{"role": "system", "content": GENERATION_SYSTEM_PROMPT},
                {"role": "user",   "content": user_prompt}]

    def call_openai(self, model_name, temperature, max_retries=3):
        """Call OpenAI API"""
        headers = {"Authorization": f"Bearer {self.api_keys['openai']}", "Content-Type": "application/json"}
        payload = {"model": model_name, "messages": self._build_payload_messages(),
                   "max_completion_tokens": self.max_comp, "reasoning_effort": "high",
                   "response_format": {"type": "json_object"}}
        for attempt in range(max_retries):
            try:
                print(f"    🔄 Calling OpenAI: {model_name} @ temp={temperature}...")
                r = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload, timeout=90)
                r.raise_for_status()
                res = r.json()
                content = res['choices'][0]['message']['content']
                tokens  = res.get('usage', {}).get('total_tokens', len(content) / 4)
                return self.parse_json_response(content, tokens)
            except Exception as e:
                print(f"    ✗ OpenAI call failed (attempt {attempt+1}/{max_retries}): {e}")
                if attempt < max_retries - 1: time.sleep(2 ** attempt)
        return None, 0

    def call_anthropic(self, model_name, temperature, max_retries=3):
        """Call Anthropic (Claude) API"""
        headers = {"x-api-key": self.api_keys['anthropic'],
                   "anthropic-version": "2023-06-01", "Content-Type": "application/json"}
        user_prompt = USER_PROMPT_TEMPLATE.format(
            project_description=self.project_description, json_schema=JSON_SCHEMA)
        payload = {"model": model_name, "max_tokens": self.max_comp,
                   "temperature": temperature, "system": GENERATION_SYSTEM_PROMPT,
                   "messages": [{"role": "user", "content": user_prompt}]}
        for attempt in range(max_retries):
            try:
                print(f"    🔄 Calling Anthropic: {model_name} @ temp={temperature}...")
                r = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=payload, timeout=90)
                r.raise_for_status()
                res = r.json()
                content = res['content'][0]['text']
                tokens  = res.get('usage', {}).get('input_tokens', 0) + res.get('usage', {}).get('output_tokens', 0)
                return self.parse_json_response(content, tokens)
            except Exception as e:
                print(f"    ✗ Anthropic call failed (attempt {attempt+1}/{max_retries}): {e}")
                if attempt < max_retries - 1: time.sleep(2 ** attempt)
        return None, 0

    def call_google(self, model_name, temperature, max_retries=3):
        """Call Google (Gemini) API"""
        user_prompt = USER_PROMPT_TEMPLATE.format(
            project_description=self.project_description, json_schema=JSON_SCHEMA)
        full_prompt = f"{GENERATION_SYSTEM_PROMPT}\n\n{user_prompt}"
        payload = {"contents": [{"parts": [{"text": full_prompt}]}],
                   "generationConfig": {"temperature": temperature, "maxOutputTokens": self.max_comp}}
        url = f"https://generativelanguage.googleapis.com/v1beta/models/{model_name}:generateContent?key={self.api_keys['google']}"
        for attempt in range(max_retries):
            try:
                print(f"    🔄 Calling Google: {model_name} @ temp={temperature}...")
                r = requests.post(url, json=payload, timeout=60)
                r.raise_for_status()
                res = r.json()
                content = res['candidates'][0]['content']['parts'][0]['text']
                tokens  = res.get('usageMetadata', {}).get('totalTokenCount', len(content) / 4)
                return self.parse_json_response(content, tokens)
            except Exception as e:
                print(f"    ✗ Google call failed (attempt {attempt+1}/{max_retries}): {e}")
                if attempt < max_retries - 1: time.sleep(2 ** attempt)
        return None, 0

    def call_deepinfra(self, model_name, temperature, max_retries=3):
        """Call DeepInfra API"""
        headers = {"Authorization": f"Bearer {self.api_keys['deepinfra']}", "Content-Type": "application/json"}
        payload = {"model": model_name, "messages": self._build_payload_messages(),
                   "temperature": temperature, "max_tokens": self.max_comp}
        for attempt in range(max_retries):
            try:
                print(f"    🔄 Calling DeepInfra: {model_name} @ temp={temperature}...")
                r = requests.post("https://api.deepinfra.com/v1/openai/chat/completions", headers=headers, json=payload, timeout=180)
                r.raise_for_status()
                res = r.json()
                content = res['choices'][0]['message']['content']
                tokens  = res.get('usage', {}).get('total_tokens', len(content) / 4)
                return self.parse_json_response(content, tokens)
            except Exception as e:
                print(f"    ✗ DeepInfra call failed (attempt {attempt+1}/{max_retries}): {e}")
                if attempt < max_retries - 1: time.sleep(2 ** attempt)
        return None, 0

    def parse_json_response(self, content, tokens_used):
        """Parse JSON from API response"""
        try:
            s = content.find('{')
            e = content.rfind('}') + 1
            if s != -1 and e > s:
                parsed = json.loads(content[s:e])
                print(f"    ✓ API call successful ({tokens_used:.0f} tokens)")
                return parsed, tokens_used
        except json.JSONDecodeError:
            print(f"    ✗ Failed to parse JSON from response")
        return None, 0

    def get_next_batch(self, model_name, temp):
        """Make a live API call if under repeat limit"""
        key = (model_name, temp)
        if self.call_counts[key] >= self.max_repeats:
            return [], 0
        result, tokens = self.call_api(model_name, temp)
        self.call_counts[key] += 1
        if result:
            items = []
            if result.get('stakeholders'):
                items += [s['role'] for s in result['stakeholders'] if s.get('role')]
            if result.get('vulnerable_groups'):
                items += [v['group'] for v in result['vulnerable_groups'] if v.get('group')]
            return items, tokens
        return [], 0


In [7]:
# ── ProjectSimulation class ───────────────────────────────────────────────────
class ProjectSimulation:
    def __init__(self, provider, project_name, cluster_config_map, config_name=''):
        self.provider  = provider
        self.pid        = project_name
        self.config_name = config_name
        self.kb_text      = []
        self.kb_embeddings = None
        self.last_cluster  = None
        self.master_log    = []
        self.consecutive_low_yields = 0
        self.RESCUE_STRIKES_LIMIT   = 3
        self.TERMINATION_LIMIT      = 5
        self.stats = {"accepted": 0, "rejected": 0, "cost": 0.0,
                      "cost_tier_0": 0.0, "cost_tier_1": 0.0, "cost_tier_2": 0.0,
                      "entropy": 0.0, "model_counts": defaultdict(int), "api_calls": 0}
        self.clusters = {}
        for cid, m_list in cluster_config_map.items():
            self.clusters[cid] = []
            for m_name in m_list:
                if m_name in MODEL_REGISTRY:
                    meta = MODEL_REGISTRY[m_name]
                    for temp, scores in meta['variations'].items():
                        self.clusters[cid].append({
                            'name': m_name, 'temp': temp, 'tier': meta['tier'],
                            'score': scores['score'], 'pure_score': scores['pure_score'],
                            'cost': meta['cost'], 'saturated': False})

    def find_highest_pure_score_model(self):
        candidates = [(cid, a, a['pure_score']) for cid, al in self.clusters.items()
                      for a in al if not a['saturated']]
        if not candidates: return None, None
        candidates.sort(key=lambda x: x[2], reverse=True)
        return candidates[0][0], candidates[0][1]

    def get_active_agent(self, cluster_id):
        valid = [a for a in self.clusters[cluster_id] if not a['saturated']]
        if not valid: return None
        return sorted(valid, key=lambda x: x['score'], reverse=True)[0]

    def get_next_move(self):
        if self.consecutive_low_yields >= self.TERMINATION_LIMIT: return None, None
        if self.consecutive_low_yields >= self.RESCUE_STRIKES_LIMIT:
            cid, best = self.find_highest_pure_score_model()
            return (cid, best) if best else (None, None)
        candidates = [(cid, a, a['score']) for cid in self.clusters
                      for a in [self.get_active_agent(cid)] if a]
        if not candidates: return None, None
        pool = [c for c in candidates if self.last_cluster is None or c[0] != self.last_cluster]
        if not pool: pool = candidates
        return sorted(pool, key=lambda x: -x[2])[0][:2]

    def normalize(self, text): return text.lower().strip()

    def process_batch(self, batch, model_name, temp, tier, sim_threshold):
        acc = rej = 0
        for item in batch:
            clean = self.normalize(item)
            if not clean: continue
            new_emb = embedding_model.encode([clean])
            status, sim_score, closest_match = 'Accepted', 0.0, None
            if self.kb_embeddings is not None:
                sims = cosine_similarity(new_emb, self.kb_embeddings)[0]
                idx  = np.argmax(sims)
                sim_score = float(sims[idx])
                closest_match  = self.kb_text[idx]
                if sim_score > sim_threshold: status = 'Rejected'
            self.master_log.append({
                'Config': self.config_name, 'Project': self.pid,
                'Model':  model_name.split('/')[-1], 'Temp': temp, 'Tier': tier,
                'Item':   clean, 'Status': status,
                'Sim':    round(sim_score, 4), 'Closest_match': closest_match})
            if status == 'Accepted':
                print(f"      ✓ {item}")
                acc += 1
                self.kb_text.append(clean)
                self.kb_embeddings = new_emb if self.kb_embeddings is None \
                                     else np.vstack([self.kb_embeddings, new_emb])
            else:
                rej += 1
        return acc, rej

    def calculate_metrics(self):
        if self.kb_embeddings is None or len(self.kb_text) < 2: return 0.0, 0.0
        dists = cosine_distances(self.kb_embeddings)
        dispersion = np.mean(dists)
        try:
            n_clusters = min(5, len(self.kb_text))
            km = KMeans(n_clusters=n_clusters, n_init='auto', random_state=42).fit(self.kb_embeddings)
            probs = np.bincount(km.labels_, minlength=n_clusters) / len(self.kb_text)
            ent = entropy(probs, base=np.e) / np.log(n_clusters)
        except: ent = 0.0
        return dispersion, ent

    def run(self, cost_cap, item_cap, yield_threshold, sim_threshold):
        iteration = 0
        while True:
            iteration += 1
            if self.stats['cost']     >= cost_cap: print(f'\n⚠ Cost cap: ${self.stats["cost"]:.4f}'); break
            if self.stats['accepted'] >= item_cap: print(f'\n⚠ Item cap: {self.stats["accepted"]}'); break
            cid, agent = self.get_next_move()
            if not agent: print('\n⚠ No more agents'); break
            m_name, m_temp = agent['name'], agent['temp']
            print(f"\n[Iter {iteration}] Cluster {cid}: {m_name.split('/')[-1]} @ temp={m_temp}")
            batch, tokens = self.provider.get_next_batch(m_name, m_temp)
            self.stats['api_calls'] += 1
            cost = (tokens / 1_000_000) * agent['cost']
            self.stats['cost'] += cost
            if agent['tier'] == 0: self.stats['cost_tier_0'] += cost
            elif agent['tier'] == 1: self.stats['cost_tier_1'] += cost
            elif agent['tier'] == 2: self.stats['cost_tier_2'] += cost
            self.stats['model_counts'][f"{m_name.split('/')[-1]}@{m_temp}"] += 1
            if not batch:
                print("    ⚠ No data — marking saturated")
                agent['saturated'] = True
                self.consecutive_low_yields += 1
            else:
                print(f"    Processing {len(batch)} items...")
                acc, rej = self.process_batch(batch, m_name, m_temp, agent['tier'], sim_threshold)
                self.stats['accepted'] += acc
                self.stats['rejected'] += rej
                print(f"    → Accepted: {acc}, Rejected: {rej}, Total: {self.stats['accepted']}, Cost: ${self.stats['cost']:.4f}")
                if acc < yield_threshold:
                    print(f"    ⚠ Low yield ({acc} < {yield_threshold}) — saturating")
                    agent['saturated'] = True
                    self.consecutive_low_yields += 1
                else:
                    self.consecutive_low_yields = 0
            self.last_cluster = cid
        d, e = self.calculate_metrics()
        self.stats['dispersion'] = d
        self.stats['entropy']    = e
        return self.stats, self.master_log


In [8]:
# ── Frequency table helpers ───────────────────────────────────────────────────
def generate_frequency_table_with_rejected(logs_df):
    """Build frequency table: each accepted role counts itself + all rejected variants mapped to it."""
    accepted_df = logs_df[logs_df['Status'] == 'Accepted'].copy()
    rejected_df = logs_df[logs_df['Status'] == 'Rejected'].copy()
    if len(accepted_df) == 0:
        print('No accepted items found.')
        return pd.DataFrame()
    print(f"\nGenerating frequency table...")
    print(f"  Accepted: {len(accepted_df)}  |  Rejected: {len(rejected_df)}")
    rejection_counts = rejected_df['Closest_match'].value_counts().to_dict()
    rows = [{'Canonical_Role': item, 'Total_Mentions': 1 + rejection_counts.get(item, 0)}
            for item in accepted_df['Item'].unique()]
    return pd.DataFrame(rows).sort_values('Total_Mentions', ascending=False).reset_index(drop=True)


# ── run_experiment ────────────────────────────────────────────────────────────
def run_experiment(project_name, project_description):
    print('='*80)
    print('LIVE API EXPERIMENT')
    print('='*80)
    print(f"\nProject        : {project_name}")
    print(f"Max repeats    : {MAX_REPEATS_PER_MODEL}")
    print(f"Cost cap       : ${COST_CAP}  |  Item cap: {ITEM_CAP}")
    print(f"Yield threshold: {YIELD_THRESHOLD}  |  Sim threshold: {SIM_THRESHOLD}")
    print('='*80)

    provider = LiveAPIProvider(API_KEYS, project_description, MAX_REPEATS_PER_MODEL)
    sim = ProjectSimulation(provider, project_name, build_cluster_config(), config_name='4_role_Clusters')
    stats, logs = sim.run(COST_CAP, ITEM_CAP, YIELD_THRESHOLD, SIM_THRESHOLD)

    runs_str = ', '.join(f'{m}: {c}' for m, c in
                         sorted(stats['model_counts'].items(), key=lambda x: x[1], reverse=True))
    print(f"\n{'='*80}")
    print('RESULTS')
    print(f"{'='*80}")
    print(f"API Calls : {stats['api_calls']}")
    print(f"Accepted  : {stats['accepted']}  |  Rejected: {stats['rejected']}")
    print(f"Total Cost: ${stats['cost']:.4f}  (T0: ${stats['cost_tier_0']:.4f}, T1: ${stats['cost_tier_1']:.4f}, T2: ${stats['cost_tier_2']:.4f})")
    print(f"Dispersion: {stats['dispersion']:.4f}  |  Entropy: {stats['entropy']:.4f}")
    print(f"Model usage: {runs_str}")

    if logs:
        logs_df = pd.DataFrame(logs)
        logs_df.to_csv(GENERATION_OUTPUT, index=False)
        print(f"\n✓ Detailed logs saved to: {GENERATION_OUTPUT}")

        freq_table = generate_frequency_table_with_rejected(logs_df)
        if not freq_table.empty:
            freq_table.to_csv(FREQ_TABLE_OUTPUT, index=False)
            print(f"✓ Frequency table saved to: {FREQ_TABLE_OUTPUT}")
            print("\nTop 20 Most Mentioned Roles:")
            print(freq_table.head(20)[['Canonical_Role', 'Total_Mentions']].to_string(index=False))

    return stats, logs


In [9]:
# ── ▶ Run generation ──────────────────────────────────────────────────────────
stats, logs = run_experiment(PROJECT_NAME, PROJECT_DESCRIPTION)


LIVE API EXPERIMENT

Project        : AI-powered student assessment
Max repeats    : 3
Cost cap       : $5.0  |  Item cap: 200
Yield threshold: 3  |  Sim threshold: 0.8

[Iter 1] Cluster 3: gpt-oss-20b @ temp=0.3
    🔄 Calling DeepInfra: openai/gpt-oss-20b @ temp=0.3...
    ✓ API call successful (6343 tokens)
    Processing 38 items...
      ✓ Student
      ✓ Parent/guardian
      ✓ Teacher
      ✓ School principal
      ✓ School district IT manager
      ✓ School district procurement officer
      ✓ School district legal counsel
      ✓ School district compliance officer
      ✓ School district risk management officer
      ✓ School district data protection officer
      ✓ School district data governance board member
      ✓ School district policy maker
      ✓ School district financial officer
      ✓ School district technology support staff
      ✓ School district community liaison
      ✓ School district student services coordinator
      ✓ School district special education coordin

## Section 4 — Stakeholder Evaluation

Scores each accepted stakeholder on Legitimacy, Power, and Urgency using DeepSeek
(Mitchell's Salience Model adapted for AI Ethics). Input: `live_api_results_detailed.csv`.
Output: `stakeholder_salience_evaluated.csv`.

In [10]:
EVALUATION_SYSTEM_PROMPT = """
You are an expert in AI Ethics and Requirements Engineering.
I will provide three inputs:
1. **System Description**
2. **Candidate Stakeholder** (The specific entity to judge)
3. **Complete List of Stakeholders** (Context for later use)

Your task is to perform a two-stage evaluation.

---

### **STEP 1: ISOLATED EVALUATION**
**INSTRUCTION:** Evaluate the Candidate Stakeholder **SOLELY** based on the System Description.
**CONSTRAINT:** Do **NOT** use the Complete List of Stakeholders for this step.

Use Mitchell's Salience Model (adapted for AI Ethics) to assess on 3 dimensions (0-5):

1. **Legitimacy** — Does this entity have valid moral/legal standing?
   - Entitlement: legal or social right to fairness, privacy, respect?
   - Desert: are they subjects of the system's decisions or data providers?
   - Rule: Vulnerable groups have HIGH Legitimacy regardless of current harm.
   - (0 = Hallucination/Invented role, 5 = High Standing/Rights-Holder)

2. **Power** — Can this entity enforce their will on the project?
   - Note: Vulnerable groups often have Low Power — this makes them Dependent Stakeholders.

3. **Urgency** — Is there a threat to the stakeholder's fundamental Needs?
   - Criticality: does the system risk bias, safety violation, or financial loss?
   - Time-sensitivity: could harm be irreversible?

**Classification Logic:**
- **Valid** if Legitimacy >= 2.
- **Hallucination** if entity does not exist or has Legitimacy 0-1.

---

### **STEP 2: CALIBRATION & COMPARISON**
**INSTRUCTION:** Now read the Complete List of Stakeholders.
- **Redundancy Check (0-5):** Is there a better/broader term already in the list?
- **Relative Importance:** Adjust scores relative to other stakeholders.

---

### **OUTPUT FORMAT (JSON)**
{
  "legitimacy_score_before_calibration": <int>,
  "power_score_before_calibration": <int>,
  "urgency_score_before_calibration": <int>,
  "reasoning_before_calibration": "Evaluated in isolation: ...",
  "classification_before_calibration": "Valid" or "Hallucination",
  "legitimacy_score_after_calibration": <int>,
  "power_score_after_calibration": <int>,
  "urgency_score_after_calibration": <int>,
  "redundancy_score_after_calibration": <int>,
  "reasoning_after_calibration": "Compared to [Role X] from the list, ...",
  "classification_after_calibration": "Valid" or "Hallucination" or "redundant"
}
"""


In [11]:
# ── call_deepseek ─────────────────────────────────────────────────────────────
def call_deepseek(system_description: str, candidate_stakeholder: str,
                  complete_stakeholder_list: List[str], max_retries: int = 3) -> Dict:
    """Call DeepSeek via DeepInfra to evaluate stakeholder salience."""
    stakeholder_list_text = '\n'.join(f'- {s}' for s in complete_stakeholder_list)
    user_prompt = f"""
**SYSTEM DESCRIPTION:**
{system_description}

**CANDIDATE STAKEHOLDER TO EVALUATE:**
{candidate_stakeholder}

**COMPLETE LIST OF STAKEHOLDERS (For Step 2 Only):**
{stakeholder_list_text}

Provide your evaluation in the exact JSON format specified.
"""
    headers = {'Authorization': f'Bearer {DEEPINFRA_API_KEY}', 'Content-Type': 'application/json'}
    payload = {'model': DEEPSEEK_MODEL,
               'messages': [{'role': 'system', 'content': EVALUATION_SYSTEM_PROMPT},
                             {'role': 'user',   'content': user_prompt}],
               'temperature': 0.3, 'max_tokens': 2000,
               'response_format': {'type': 'json_object'}}
    for attempt in range(max_retries):
        try:
            print(f"  🔄 Evaluating '{candidate_stakeholder}' (attempt {attempt+1}/{max_retries})...")
            r = requests.post(DEEPINFRA_BASE_URL, headers=headers, json=payload, timeout=120)
            r.raise_for_status()
            parsed = json.loads(r.json()['choices'][0]['message']['content'])
            print('  ✓ Success')
            return parsed
        except requests.exceptions.Timeout:
            print(f"  ✗ Timeout on attempt {attempt+1}")
        except requests.exceptions.HTTPError as e:
            print(f"  ✗ HTTP error on attempt {attempt+1}: {e}")
        except json.JSONDecodeError as e:
            print(f"  ✗ JSON parse error on attempt {attempt+1}: {e}")
        except Exception as e:
            print(f"  ✗ Unexpected error on attempt {attempt+1}: {e}")
        if attempt < max_retries - 1: time.sleep(2 ** attempt)
    return {"error": "max_retries_exceeded"}


In [12]:
# ── evaluate_stakeholders ─────────────────────────────────────────────────────
def evaluate_stakeholders(input_csv: str, output_csv: str):
    """Score all accepted stakeholders on Legitimacy, Power, Urgency."""
    print('='*80)
    print('STAKEHOLDER SALIENCE EVALUATION')
    print('='*80)

    print(f"\n📂 Loading: {input_csv}")
    df = pd.read_csv(input_csv)
    df_accepted = df[df['Status'] == 'Accepted'].copy()
    print(f"✓ Loaded {len(df)} rows | {len(df_accepted)} accepted stakeholders")

    new_cols = ['legitimacy_before', 'power_before', 'urgency_before',
                'reasoning_before', 'classification_before',
                'legitimacy_after', 'power_after', 'urgency_after', 'redundancy_after',
                'reasoning_after', 'classification_after', 'evaluation_error']
    for col in new_cols:
        df_accepted[col] = None

    projects = df_accepted.groupby('Project')
    print(f"\n🔍 {len(projects)} unique projects to evaluate")

    for proj_idx, (project_name, project_group) in enumerate(projects, 1):
        print(f"\n{'='*80}")
        print(f"PROJECT {proj_idx}/{len(projects)}: {project_name}")
        print(f"{'='*80}")
        stakeholder_list = project_group['Item'].tolist()
        print(f"📋 {len(stakeholder_list)} stakeholders")

        for idx, row in project_group.iterrows():
            candidate  = row['Item']
            pos        = stakeholder_list.index(candidate)
            so_far     = stakeholder_list[:pos + 1]
            print(f"\n  [{pos+1}/{len(stakeholder_list)}] {candidate}")
            evaluation = call_deepseek(project_name, candidate, so_far)

            if 'error' in evaluation:
                df_accepted.at[idx, 'evaluation_error'] = evaluation['error']
                print(f" Error: {evaluation['error']}")
            else:
                df_accepted.at[idx, 'legitimacy_before']     = evaluation.get('legitimacy_score_before_calibration')
                df_accepted.at[idx, 'power_before']          = evaluation.get('power_score_before_calibration')
                df_accepted.at[idx, 'urgency_before']        = evaluation.get('urgency_score_before_calibration')
                df_accepted.at[idx, 'reasoning_before']      = evaluation.get('reasoning_before_calibration')
                df_accepted.at[idx, 'classification_before'] = evaluation.get('classification_before_calibration')
                df_accepted.at[idx, 'legitimacy_after']      = evaluation.get('legitimacy_score_after_calibration')
                df_accepted.at[idx, 'power_after']           = evaluation.get('power_score_after_calibration')
                df_accepted.at[idx, 'urgency_after']         = evaluation.get('urgency_score_after_calibration')
                df_accepted.at[idx, 'redundancy_after']      = evaluation.get('redundancy_score_after_calibration')
                df_accepted.at[idx, 'reasoning_after']       = evaluation.get('reasoning_after_calibration')
                df_accepted.at[idx, 'classification_after']  = evaluation.get('classification_after_calibration')
                print(f"  ✓ Before: L={evaluation.get('legitimacy_score_before_calibration')} "
                      f"P={evaluation.get('power_score_before_calibration')} "
                      f"U={evaluation.get('urgency_score_before_calibration')} → "
                      f"{evaluation.get('classification_before_calibration')}")
                print(f"  ✓ After:  L={evaluation.get('legitimacy_score_after_calibration')} "
                      f"P={evaluation.get('power_score_after_calibration')} "
                      f"U={evaluation.get('urgency_score_after_calibration')} "
                      f"R={evaluation.get('redundancy_score_after_calibration')} → "
                      f"{evaluation.get('classification_after_calibration')}")
            time.sleep(0.5)

    df_accepted.to_csv(output_csv, index=False)
    print(f"\n✓ Saved {len(df_accepted)} evaluated stakeholders to: {output_csv}")

    errors = df_accepted['evaluation_error'].notna().sum()
    print(f"Errors: {errors}")
    if 'classification_after' in df_accepted.columns:
        for cls, cnt in df_accepted['classification_after'].value_counts().items():
            print(f"  {cls}: {cnt} ({cnt/len(df_accepted)*100:.1f}%)")

    return df_accepted


In [13]:
# ── ▶ Run evaluation ──────────────────────────────────────────────────────────
results_df = evaluate_stakeholders(GENERATION_OUTPUT, EVALUATION_OUTPUT)


STAKEHOLDER SALIENCE EVALUATION

📂 Loading: live_api_results_detailed.csv
✓ Loaded 796 rows | 159 accepted stakeholders

🔍 1 unique projects to evaluate

PROJECT 1/1: AI-powered student assessment
📋 159 stakeholders

  [1/159] student
  🔄 Evaluating 'student' (attempt 1/3)...
  ✓ Success
  ✓ Before: L=5 P=2 U=4 → Valid
  ✓ After:  L=5 P=2 U=4 R=0 → Valid

  [2/159] parent/guardian
  🔄 Evaluating 'parent/guardian' (attempt 1/3)...
  ✓ Success
  ✓ Before: L=5 P=3 U=4 → Valid
  ✓ After:  L=5 P=3 U=4 R=0 → Valid

  [3/159] teacher
  🔄 Evaluating 'teacher' (attempt 1/3)...
  ✓ Success
  ✓ Before: L=5 P=4 U=4 → Valid
  ✓ After:  L=5 P=4 U=4 R=0 → Valid

  [4/159] school principal
  🔄 Evaluating 'school principal' (attempt 1/3)...
  ✓ Success
  ✓ Before: L=5 P=5 U=4 → Valid
  ✓ After:  L=5 P=5 U=4 R=0 → Valid

  [5/159] school district it manager
  🔄 Evaluating 'school district it manager' (attempt 1/3)...
  ✓ Success
  ✓ Before: L=4 P=5 U=3 → Valid
  ✓ After:  L=4 P=5 U=3 R=0 → Valid

  [6/1

## Section 5 — Clustering

Combines sentence embeddings with salience scores to cluster stakeholders.
Representatives are selected by **frequency** (most-mentioned role per cluster).
Input: `stakeholder_salience_evaluated.csv` + `frequency_table_with_rejected.csv`.
Output: `stakeholder_clustered.csv`.

In [47]:
# ── Clustering utilities ──────────────────────────────────────────────────────

def mean_pooling(last_hidden, attention_mask):
    mask   = attention_mask.unsqueeze(-1).expand(last_hidden.size()).float()
    summed = torch.sum(last_hidden * mask, dim=1)
    return summed / torch.clamp(mask.sum(dim=1), min=1e-9)


@torch.no_grad()
def embed_texts(texts, model_name=EMBEDDING_MODEL, batch_size=BATCH_SIZE):
    """Generate normalized sentence embeddings."""
    device    = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model     = AutoModel.from_pretrained(model_name).to(device)
    model.eval()
    all_vecs  = []
    for i in tqdm(range(0, len(texts), batch_size), desc='Embedding'):
        batch   = texts[i:i + batch_size]
        encoded = tokenizer(batch, padding=True, truncation=True,
                            max_length=512, return_tensors='pt').to(device)
        out     = model(**encoded)
        vec     = mean_pooling(out.last_hidden_state, encoded['attention_mask'])
        vec     = torch.nn.functional.normalize(vec, p=2, dim=1)
        all_vecs.append(vec.cpu().numpy())
    return np.vstack(all_vecs)


def build_embed_text(project, stakeholder, reasoning_before, reasoning_after):
    return (f'Stakeholder: {stakeholder}\n'
            f'Reasoning (Isolated): {reasoning_before}\n'
            f'Reasoning (Calibrated): {reasoning_after}\n'
            f'Project Context: {project}\n')


def mean_intra_cluster_distance(X, labels):
    """Mean cosine distance from each point to its cluster centroid (lower = more compact)."""
    dists = []
    for cid in np.unique(labels):
        pts = X[labels == cid]
        if len(pts) <= 1: continue
        centroid = pts.mean(axis=0, keepdims=True)
        centroid = centroid / (np.linalg.norm(centroid) + 1e-9)
        dists.append(float(1.0 - (pts @ centroid.T).mean()))
    return float(np.mean(dists)) if dists else 0.0


def mean_score_variance(scores, labels):
    """Mean variance of salience scores within clusters (lower = more homogeneous)."""
    variances = []
    for cid in np.unique(labels):
        cs = scores[labels == cid]
        if len(cs) <= 1: continue
        variances.append(np.mean(np.var(cs, axis=0)))
    return float(np.mean(variances)) if variances else 0.0



def cluster_with_homogeneity_stop(X, scores,
                                   k_min=K_MIN, k_max=K_MAX, step=K_STEP,
                                   dist_eps=DIST_EPS, var_eps=VAR_EPS, patience=PATIENCE):
    """Adaptive k-means: stop when both compactness & score-variance improvements plateau."""
    prev_dist = prev_var = best_result = best_k = best_obj = None
    streak = 0
    for k in range(k_min, min(k_max, len(X)) + 1, step):
        km     = KMeans(n_clusters=k, random_state=42, n_init=30, max_iter=300)
        labels = km.fit_predict(X)
        intra  = mean_intra_cluster_distance(X, labels)
        s_var  = mean_score_variance(scores, labels)
        if prev_dist is not None:
            if (prev_dist - intra < dist_eps) and (prev_var - s_var < var_eps):
                streak += 1
            else:
                streak = 0
            if streak >= patience:
                print(f"  Early stopping at k={k} | best k={best_k}")
                break
        obj = intra + s_var
        if best_obj is None or obj < best_obj:
            best_obj, best_k, best_result = obj, k, (labels, k, intra, s_var)
        prev_dist, prev_var = intra, s_var
    print(f"  Best k={best_k} | objective={best_obj:.6f} (compactness={best_result[2]:.6f}, score_var={best_result[3]:.6f})")
    return best_result


In [74]:
# ── RHRC clustering ───────────────────────────────────────────────────────────
def _cluster_objective(X, scores, index_groups):
    """
    Combined objective across a set of clusters (list of index arrays).
    Analogous to the regression MAE in the paper — lower is better.
    """
    dist_vals, var_vals = [], []
    for idx in index_groups:
        pts = X[idx]
        if len(pts) <= 1:
            continue
        centroid = pts.mean(axis=0, keepdims=True)
        centroid = centroid / (np.linalg.norm(centroid) + 1e-9)
        dist_vals.append(float(1.0 - (pts @ centroid.T).mean()))
        var_vals.append(float(np.mean(np.var(scores[idx], axis=0))))
    d = float(np.mean(dist_vals)) if dist_vals else 0.0
    v = float(np.mean(var_vals))  if var_vals  else 0.0
    return d + v


def rhrc_cluster(X, scores,
                 min_child=MIN_CHILD_SIZE,
                 min_parent=MIN_PARENT_SIZE,
                 patience=PATIENCE):
    """
    Recursive Hierarchical Regression Clustering adapted for stakeholder embeddings.

    Binary splits (k=2) are applied top-down, recursing on each child.
    Three stopping conditions mirror the RHRC paper:
      ST1 — any child cluster would have fewer than min_child points
      ST2 — parent has fewer than min_parent points
      ST3 — splitting worsens the combined objective (compactness + score variance)
             → patience controls how many levels we continue past an ST3 signal

    Returns: (labels, k_used, final_compactness, final_score_var)
    """
    n          = len(X)
    labels     = np.full(n, -1, dtype=int)
    next_label = [0]   # mutable counter shared across recursion

    def assign(idx):
        """Seal a cluster — give all its points a fresh label."""
        lbl = next_label[0]
        next_label[0] += 1
        labels[idx] = lbl

    def recurse(idx, pat_left, depth=0):
        indent = '  ' + '  ' * depth

        # ── ST2: parent too small to split ────────────────────────────────────
        if len(idx) < min_parent:
            print(f"{indent}ST2 → seal ({len(idx)} pts)")
            assign(idx)
            return

        # ── Attempt binary split ──────────────────────────────────────────────
        km           = KMeans(n_clusters=2, init='k-means++',
                              n_init=10, max_iter=300, random_state=42)
        child_labels = km.fit_predict(X[idx])
        idx_L        = idx[child_labels == 0]
        idx_R        = idx[child_labels == 1]

        # ── ST1: a child is too small ─────────────────────────────────────────
        if len(idx_L) < min_child or len(idx_R) < min_child:
            print(f"{indent}ST1 → seal ({len(idx_L)}/{len(idx_R)} pts)")
            assign(idx)
            return

        # ── ST3: does splitting improve the objective? ────────────────────────
        parent_obj   = _cluster_objective(X, scores, [idx])
        children_obj = _cluster_objective(X, scores, [idx_L, idx_R])

        if children_obj >= parent_obj:
            if pat_left <= 0:
                # Patience exhausted — accept the stop
                print(f"{indent}ST3 → seal (obj {parent_obj:.5f} → {children_obj:.5f}, patience=0)")
                assign(idx)
                return
            else:
                # Burn one patience token and keep going (paper's patience mechanism)
                print(f"{indent}ST3 (patience {pat_left}→{pat_left-1}) obj {parent_obj:.5f} → {children_obj:.5f}")
                recurse(idx_L, pat_left - 1, depth + 1)
                recurse(idx_R, pat_left - 1, depth + 1)
                return

        # ── Split improved — recurse on both children, reset patience ─────────
        print(f"{indent}split {len(idx)} → {len(idx_L)}+{len(idx_R)}  obj {parent_obj:.5f}→{children_obj:.5f}")
        recurse(idx_L, patience, depth + 1)
        recurse(idx_R, patience, depth + 1)

    # ── Run ───────────────────────────────────────────────────────────────────
    recurse(np.arange(n), patience, depth=0)

    assert (labels >= 0).all(), "BUG: some points were never assigned a label"

    k_used = next_label[0]
    intra  = mean_intra_cluster_distance(X, labels)
    s_var  = mean_score_variance(scores, labels)
    print(f"  ✓ RHRC complete | k={k_used} | compactness={intra:.6f} | score_var={s_var:.6f}")
    return labels, k_used, intra, s_var

In [75]:
# ── cluster_stakeholders ──────────────────────────────────────────────────────
def cluster_stakeholders_ikmean(input_csv: str, freq_csv: str, output_csv: str):
    """Cluster stakeholders and select representatives by frequency."""
    print('='*80)
    print('STAKEHOLDER CLUSTERING')
    print('='*80)

    # Load data
    print(f"\n📂 Loading: {input_csv}")
    df = pd.read_csv(input_csv)
    if 'evaluation_error' in df.columns:
        err = df['evaluation_error'].notna().sum()
        if err: print(f" Removing {err} rows with evaluation errors")
        df = df[df['evaluation_error'].isna()].copy()
    print(f"✓ {len(df)} valid stakeholders")

    # Load frequency table
    print(f"📂 Loading frequency table: {freq_csv}")
    freq_map = pd.read_csv(freq_csv).set_index('Canonical_Role')['Total_Mentions'].to_dict()
    print(f"✓ {len(freq_map)} canonical roles in frequency table")

    # Step 1: Embeddings
    print(f"\n{'='*80}\nSTEP 1: EMBEDDINGS\n{'='*80}")
    embed_texts_list = [
        build_embed_text(
            project          = PROJECT_DESCRIPTION,
            stakeholder      = row['Item'],
            reasoning_before = row['reasoning_before'] if pd.notna(row['reasoning_before']) else '',
            reasoning_after  = row['reasoning_after']  if pd.notna(row['reasoning_after'])  else ''
        ) for _, row in df.iterrows()
    ]
    E = embed_texts(embed_texts_list, batch_size=BATCH_SIZE)
    print(f"✓ Embeddings shape: {E.shape}")
    np.save(EMBEDDING_OUTPUT, E)

    # Step 2: Salience scores
    print(f"\n{'='*80}\nSTEP 2: SALIENCE SCORES\n{'='*80}")
    scores = df[['legitimacy_after', 'power_before', 'urgency_after']].values.astype(np.float32)
    scores = (scores - scores.mean(axis=0, keepdims=True)) / (scores.std(axis=0, keepdims=True) + 1e-6)
    scores = scores * SCORE_WEIGHT
    print(f"✓ Scores normalized (weight={SCORE_WEIGHT})")

    # Step 3: Combined features
    F = np.hstack([E, scores]).astype(np.float32)
    F = F / (np.linalg.norm(F, axis=1, keepdims=True) + 1e-9)

    # Step 4: Cluster by project
    print(f"\n{'='*80}\nSTEP 4: CLUSTERING\n{'='*80}")
    clustered_rows = []
    for proj_idx, project in enumerate(df['Project'].unique(), 1):
        print(f"\n[{proj_idx}] Project: {project}")
        pmask      = df['Project'] == project
        project_df = df[pmask].copy().reset_index(drop=True)
        X          = F[np.where(pmask)[0]]
        score_mat  = project_df[['legitimacy_after', 'power_before', 'urgency_after']].values
        print(f"  Stakeholders: {len(project_df)}")

        labels, k_used, compactness, score_var = cluster_with_homogeneity_stop(
            X, score_mat,
            k_min=min(K_MIN, len(project_df) - 1),
            k_max=min(K_MAX, len(project_df)),
            step=K_STEP)

        project_df['Cluster']            = labels
        project_df['K_Used']             = k_used
        project_df['Cluster_Compactness'] = compactness
        project_df['Cluster_ScoreVar']   = score_var

        # Step 5: Select representatives by frequency
        global_max_freq   = max(freq_map.values()) if freq_map else 1
        freq_threshold    = global_max_freq / 4

        project_df['Is_Representative'] = False
        project_df['Frequency'] = project_df['Item'].map(freq_map).fillna(0).astype(int)

        for cluster_id in project_df['Cluster'].unique():
            cluster_df = project_df[project_df['Cluster'] == cluster_id].sort_values(
                by='Frequency', ascending=False
            )

            # Always promote the highest-frequency item
            top_rep = cluster_df.iloc[0]
            project_df.loc[top_rep.name, 'Is_Representative'] = True

            # Also promote any others that clear the global threshold
            above_threshold = cluster_df[cluster_df['Frequency'] > freq_threshold]
            project_df.loc[above_threshold.index, 'Is_Representative'] = True

        clustered_rows.append(project_df)

    # Step 6: Save
    final_df = pd.concat(clustered_rows, ignore_index=True)
    final_df.to_csv(output_csv, index=False)
    print(f"\n✓ Saved to: {output_csv}")
    print(f"Total stakeholders : {len(final_df)}")
    rep_count = project_df['Is_Representative'].sum()
    print(f"  ✓ {k_used} clusters | {rep_count} representatives (threshold={freq_threshold:.1f})")
        # print(f"  ✓ {k_used} clusters | {project_df['Is_Representative'].sum()} representatives")

    print(f"Total representatives: {final_df['Is_Representative'].sum()}")
    return final_df


In [76]:
def cluster_stakeholders_rhrc(input_csv: str, freq_csv: str, output_csv: str):
    """Cluster stakeholders using RHRC and select representatives by frequency."""
    print('='*80)
    print('STAKEHOLDER CLUSTERING (RHRC)')
    print('='*80)

    print(f"\n📂 Loading: {input_csv}")
    df = pd.read_csv(input_csv)
    if 'evaluation_error' in df.columns:
        err = df['evaluation_error'].notna().sum()
        if err: print(f"Removing {err} rows with evaluation errors")
        df = df[df['evaluation_error'].isna()].copy()
    print(f"✓ {len(df)} valid stakeholders")

    print(f"📂 Loading frequency table: {freq_csv}")
    freq_map = pd.read_csv(freq_csv).set_index('Canonical_Role')['Total_Mentions'].to_dict()
    print(f"✓ {len(freq_map)} canonical roles in frequency table")

    # Step 1: Embeddings
    print(f"\n{'='*80}\nSTEP 1: EMBEDDINGS\n{'='*80}")
    embed_texts_list = [
        build_embed_text(
            project          = PROJECT_DESCRIPTION,
            stakeholder      = row['Item'],
            reasoning_before = row['reasoning_before'] if pd.notna(row['reasoning_before']) else '',
            reasoning_after  = row['reasoning_after']  if pd.notna(row['reasoning_after'])  else ''
        ) for _, row in df.iterrows()
    ]
    E = embed_texts(embed_texts_list, batch_size=BATCH_SIZE)
    print(f"✓ Embeddings shape: {E.shape}")
    np.save(EMBEDDING_OUTPUT, E)

    # Step 2: Salience scores
    print(f"\n{'='*80}\nSTEP 2: SALIENCE SCORES\n{'='*80}")
    scores = df[['legitimacy_after', 'power_before', 'urgency_after']].values.astype(np.float32)

    # ── Normalize each score dimension independently before combining ──────────
    # This prevents score variance from dominating over compactness in the
    # RHRC objective, since raw scores (0–5) and cosine distances (0–1) differ
    # in scale. After normalization both sit in the same z-score space.
    scores_norm = (scores - scores.mean(axis=0, keepdims=True)) / \
                  (scores.std(axis=0, keepdims=True) + 1e-6)
    scores_norm = scores_norm * SCORE_WEIGHT
    print(f"✓ Scores normalized (weight={SCORE_WEIGHT})")

    # Step 3: Combined features
    F = np.hstack([E, scores_norm]).astype(np.float32)
    F = F / (np.linalg.norm(F, axis=1, keepdims=True) + 1e-9)

    # Step 4: Cluster by project using RHRC
    print(f"\n{'='*80}\nSTEP 4: RHRC CLUSTERING\n{'='*80}")
    print(f"  min_child={MIN_CHILD_SIZE} | min_parent={MIN_PARENT_SIZE} | patience={PATIENCE}")
    clustered_rows = []

    for proj_idx, project in enumerate(df['Project'].unique(), 1):
        print(f"\n[{proj_idx}] Project: {project}")
        pmask      = df['Project'] == project
        project_df = df[pmask].copy().reset_index(drop=True)
        X          = F[np.where(pmask)[0]]
        score_mat  = scores_norm[np.where(pmask)[0]]   # pass normalized scores to RHRC
        print(f"  Stakeholders: {len(project_df)}")

        labels, k_used, compactness, score_var = rhrc_cluster(
            X, score_mat,
            min_child  = MIN_CHILD_SIZE,
            min_parent = MIN_PARENT_SIZE,
            patience   = PATIENCE
        )

        project_df['Cluster']             = labels
        project_df['K_Used']              = k_used
        project_df['Cluster_Compactness'] = compactness
        project_df['Cluster_ScoreVar']    = score_var

        # Step 5: Select representatives by frequency
        global_max_freq = max(freq_map.values()) if freq_map else 1
        freq_threshold  = global_max_freq / 4

        project_df['Is_Representative'] = False
        project_df['Frequency'] = project_df['Item'].map(freq_map).fillna(0).astype(int)

        for cluster_id in project_df['Cluster'].unique():
            cluster_df = project_df[project_df['Cluster'] == cluster_id].sort_values(
                by='Frequency', ascending=False
            )
            # Always promote the top-frequency item
            project_df.loc[cluster_df.iloc[0].name, 'Is_Representative'] = True
            # Also promote anyone above the global threshold
            above = cluster_df[cluster_df['Frequency'] > freq_threshold]
            project_df.loc[above.index, 'Is_Representative'] = True

        rep_count = project_df['Is_Representative'].sum()
        print(f"  ✓ {k_used} clusters | {rep_count} representatives (threshold={freq_threshold:.1f})")
        clustered_rows.append(project_df)

    # Step 6: Save
    final_df = pd.concat(clustered_rows, ignore_index=True)
    final_df.to_csv(output_csv, index=False)
    print(f"\n✓ Saved to: {output_csv}")
    print(f"Total stakeholders   : {len(final_df)}")
    print(f"Total clusters       : {final_df['K_Used'].iloc[0]}")
    print(f"Total representatives: {final_df['Is_Representative'].sum()}")
    return final_df

In [77]:
# ── ▶ Run clustering ──────────────────────────────────────────────────────────
clustered_df = cluster_stakeholders_rhrc(EVALUATION_OUTPUT, FREQ_TABLE_OUTPUT, CLUSTERED_OUTPUT)


STAKEHOLDER CLUSTERING (RHRC)

📂 Loading: stakeholder_salience_evaluated.csv
✓ 159 valid stakeholders
📂 Loading frequency table: frequency_table_with_rejected.csv
✓ 159 canonical roles in frequency table

STEP 1: EMBEDDINGS
Using device: cpu


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Embedding: 100%|██████████| 3/3 [00:23<00:00,  7.72s/it]


✓ Embeddings shape: (159, 384)

STEP 2: SALIENCE SCORES
✓ Scores normalized (weight=0.3)

STEP 4: RHRC CLUSTERING
  min_child=3 | min_parent=6 | patience=1

[1] Project: AI-powered student assessment
  Stakeholders: 159
  split 159 → 74+85  obj 0.32523→0.21785
    split 74 → 47+27  obj 0.17689→0.13360
      split 47 → 26+21  obj 0.12193→0.10003
        split 26 → 16+10  obj 0.08174→0.07268
          split 16 → 4+12  obj 0.07254→0.04801
            ST2 → seal (4 pts)
            split 12 → 4+8  obj 0.07122→0.05553
              ST2 → seal (4 pts)
              split 8 → 3+5  obj 0.06263→0.04466
                ST2 → seal (3 pts)
                ST2 → seal (5 pts)
          ST1 → seal (2/8 pts)
        split 21 → 4+17  obj 0.11831→0.07221
          ST2 → seal (4 pts)
          split 17 → 3+14  obj 0.11836→0.07484
            ST2 → seal (3 pts)
            split 14 → 11+3  obj 0.11516→0.09728
              split 11 → 4+7  obj 0.09508→0.07761
                ST2 → seal (4 pts)
            

In [80]:
# ── Stakeholder taxonomy ──────────────────────────────────────────────────────
STAKEHOLDER_TAXONOMY = {
    1:  "System Users",
    2:  "Data Subjects",
    3:  "Underrepresented Groups",
    4:  "Bystanders",
    5:  "Vulnerable Stakeholders",
    6:  "Development Teams",
    7:  "Domain Experts",
    8:  "Organizations",
    9:  "Governmental or Regulatory Bodies",
    10: "Research & Scientific Community",
    11: "Civil Society",
}

TAXONOMY_DEFINITIONS = """
1.  System Users: Individuals or organizations that directly interact with the AI system to obtain outputs or decisions.
2.  Data Subjects: Individuals whose personal or contextual data is collected, processed, or used to train, validate, or operate the AI system.
3.  Underrepresented Groups: Populations that are systematically underrepresented, misrepresented, or excluded in training data, design assumptions, or evaluation processes.
4.  Bystanders: Individuals who do not directly use or provide data to the system but may be affected during its deployment or unintentionally by its decisions.
5.  Vulnerable Stakeholders: Individuals or groups with high vulnerabilities to the harms of the system due to age, disability, socioeconomic status, legal dependency, or limited capacity to understand or contest AI decisions.
6.  Development Teams: Professionals involved in the design, development, deployment, testing, and maintenance of the AI system, including engineers, data scientists, UI/UX designers, and product owners.
7.  Domain Experts: Subject-matter experts with professional or disciplinary knowledge relevant to the system's application domain.
8.  Organizations: Entities that own, fund, or commercially benefit from the AI system.
9.  Governmental or Regulatory Bodies: Bodies responsible for oversight, compliance verification, compliance enforcement, and public accountability related to AI systems.
10. Research & Scientific Community: Researchers, academics, and professional bodies who study, critique, benchmark, or develop frameworks for AI or the application under development.
11. Civil Society: Non-governmental organizations and advocacy groups that represent societal values, often acting through awareness-raising, critique, or soft-power mechanisms.
"""

TAXONOMY_SYSTEM_PROMPT = f"""
You are an expert in AI Ethics and Stakeholder Analysis.
Given a stakeholder role and a system description, assign the 2-3 MOST RELEVANT labels
from the taxonomy below. Only assign a label if it clearly and strongly applies.
Do not assign more than 3 labels. Prioritize specificity over coverage.

TAXONOMY:
{TAXONOMY_DEFINITIONS}

OUTPUT FORMAT (JSON only, no preamble):
{{
  "labels": [<int>, ...],
  "reasoning": "One sentence explaining why each label was chosen."
}}
"""


# ── API call ──────────────────────────────────────────────────────────────────
def call_deepseek_taxonomy(stakeholder: str, system_description: str,
                            max_retries: int = 3) -> Dict:
    """Call DeepSeek to assign taxonomy labels to a single stakeholder."""
    user_prompt = f"""
**SYSTEM DESCRIPTION:**
{system_description}

**STAKEHOLDER TO LABEL:**
{stakeholder}

Assign the 2-3 most relevant labels from the taxonomy. Return JSON only.
"""
    headers = {
        'Authorization': f'Bearer {DEEPINFRA_API_KEY}',
        'Content-Type':  'application/json'
    }
    payload = {
        'model':           DEEPSEEK_MODEL,
        'messages':        [
            {'role': 'system', 'content': TAXONOMY_SYSTEM_PROMPT},
            {'role': 'user',   'content': user_prompt}
        ],
        'temperature':     0.1,   # low — this is a classification task
        'max_tokens':      300,
        'response_format': {'type': 'json_object'}
    }

    for attempt in range(max_retries):
        try:
            print(f"    🔄 Labelling '{stakeholder}' (attempt {attempt+1})...")
            r = requests.post(DEEPINFRA_BASE_URL, headers=headers,
                              json=payload, timeout=60)
            r.raise_for_status()
            parsed = json.loads(r.json()['choices'][0]['message']['content'])

            # Validate: labels must be ints in 1–11, max 3
            raw_labels = parsed.get('labels', [])
            valid = [int(l) for l in raw_labels if int(l) in STAKEHOLDER_TAXONOMY][:3]
            if not valid:
                raise ValueError(f"No valid labels in response: {raw_labels}")

            print(f"    ✓ Labels: {[STAKEHOLDER_TAXONOMY[l] for l in valid]}")
            return {
                'labels':    valid,
                'label_names': [STAKEHOLDER_TAXONOMY[l] for l in valid],
                'reasoning': parsed.get('reasoning', '')
            }

        except Exception as e:
            print(f"    ✗ Attempt {attempt+1} failed: {e}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)

    return {'labels': [], 'label_names': [], 'reasoning': 'error'}


# ── Main labelling function ───────────────────────────────────────────────────
def label_cluster_representatives(input_csv: str, output_csv: str,
                                   system_description: str = PROJECT_DESCRIPTION) -> pd.DataFrame:
    """
    Load clustered CSV, call DeepSeek to multilabel every representative,
    and write results back to output_csv with new columns:
      - taxonomy_labels      : list of ints  e.g. [1, 5]
      - taxonomy_label_names : list of strs  e.g. ['System Users', 'Vulnerable Stakeholders']
      - taxonomy_reasoning   : one-sentence justification
    """
    print('='*80)
    print('STAKEHOLDER TAXONOMY LABELLING')
    print('='*80)

    df = pd.read_csv(input_csv)
    reps = df[df['Is_Representative'] == True].copy()
    print(f"✓ Loaded {len(df)} stakeholders | {len(reps)} representatives to label")

    # Initialise columns on the full dataframe
    df['taxonomy_labels']      = None
    df['taxonomy_label_names'] = None
    df['taxonomy_reasoning']   = None

    total = len(reps)
    for i, (idx, row) in enumerate(reps.iterrows(), 1):
        print(f"\n[{i}/{total}] Cluster {row['Cluster']} — {row['Item']}")

        result = call_deepseek_taxonomy(
            stakeholder        = row['Item'],
            system_description = system_description
        )

        df.at[idx, 'taxonomy_labels']      = json.dumps(result['labels'])
        df.at[idx, 'taxonomy_label_names'] = json.dumps(result['label_names'])
        df.at[idx, 'taxonomy_reasoning']   = result['reasoning']

        time.sleep(0.3)   # gentle rate limiting

    df.to_csv(output_csv, index=False)
    print(f"\n{'='*80}")
    print(f"✓ Saved labelled results to: {output_csv}")

    # ── Summary ───────────────────────────────────────────────────────────────
    from collections import Counter
    all_labels = []
    for v in df['taxonomy_label_names'].dropna():
        all_labels.extend(json.loads(v))

    print(f"\nLabel distribution across {total} representatives:")
    for label, count in Counter(all_labels).most_common():
        bar = '█' * count
        print(f"  {label:<35} {bar} ({count})")

    return df


# ── ▶ Run labelling ───────────────────────────────────────────────────────────
LABELLED_OUTPUT = 'stakeholder_labelled.csv'
labelled_df = label_cluster_representatives(CLUSTERED_OUTPUT, LABELLED_OUTPUT)

STAKEHOLDER TAXONOMY LABELLING
✓ Loaded 159 stakeholders | 40 representatives to label

[1/40] Cluster 8 — student
    🔄 Labelling 'student' (attempt 1)...
    ✓ Labels: ['System Users', 'Data Subjects', 'Vulnerable Stakeholders']

[2/40] Cluster 10 — parent/guardian
    🔄 Labelling 'parent/guardian' (attempt 1)...
    ✓ Labels: ['Vulnerable Stakeholders', 'Data Subjects']

[3/40] Cluster 26 — teacher
    🔄 Labelling 'teacher' (attempt 1)...
    ✓ Labels: ['System Users', 'Domain Experts']

[4/40] Cluster 26 — school principal
    🔄 Labelling 'school principal' (attempt 1)...
    ✓ Labels: ['Organizations']

[5/40] Cluster 21 — school district legal counsel
    🔄 Labelling 'school district legal counsel' (attempt 1)...
    ✓ Labels: ['Governmental or Regulatory Bodies', 'Organizations']

[6/40] Cluster 30 — school district compliance officer
    🔄 Labelling 'school district compliance officer' (attempt 1)...
    ✓ Labels: ['Governmental or Regulatory Bodies', 'Organizations']

[7/40] C

## Section 6 — Visualization

Interactive cluster explorer with bubble chart, split view, and grid view.
Input: `stakeholder_clustered.csv`.

In [78]:
# ── Build cluster dictionary from CSV ─────────────────────────────────────────
df_viz = pd.read_csv(CLUSTERED_OUTPUT)

clusters = {}
for cluster_id in sorted(df_viz['Cluster'].unique()):
    cdf      = df_viz[df_viz['Cluster'] == cluster_id]
    rep_rows = cdf[cdf['Is_Representative'] == True]
    mem_rows = cdf[cdf['Is_Representative'] == False]

    reps = [
        {
            'name':       row['Item'],
            'legitimacy': float(row['legitimacy_after']) if pd.notna(row.get('legitimacy_after')) else 0,
            'power':      float(row['power_after'])      if pd.notna(row.get('power_after'))      else 0,
            'urgency':    float(row['urgency_after'])     if pd.notna(row.get('urgency_after'))     else 0,
            'frequency':  int(row.get('Frequency', 0)),
        }
        for _, row in rep_rows.iterrows()
    ]

    # Primary rep = highest frequency among representatives (for bubble chart axis values)
    primary = max(reps, key=lambda r: r['frequency'])

    clusters[int(cluster_id)] = {
        'reps':       reps,                        # full list of representatives
        'primary':    primary['name'],             # top rep (for labels/bubble positioning)
        'members':    mem_rows['Item'].tolist(),   # non-representative members only
        'legitimacy': primary['legitimacy'],       # primary rep scores (for bubble chart)
        'power':      primary['power'],
        'urgency':    primary['urgency'],
        'frequency':  primary['frequency'],
        'size':       len(cdf)
    }

print(f'✓ Loaded {len(clusters)} clusters | {len(df_viz)} stakeholders')
print(f'  Total representatives : {df_viz["Is_Representative"].sum()}')
print(f'  Avg reps per cluster  : {df_viz["Is_Representative"].sum() / len(clusters):.1f}')

✓ Loaded 32 clusters | 159 stakeholders
  Total representatives : 40
  Avg reps per cluster  : 1.2


In [79]:
STYLE = '''
<style>
@import url('https://fonts.googleapis.com/css2?family=IBM+Plex+Mono:wght@300;400;500&family=IBM+Plex+Sans:wght@400;600&display=swap');
#sk-root *{box-sizing:border-box;margin:0;padding:0}
#sk-root{font-family:'IBM Plex Mono',monospace;background:#0f1117;color:#e0e6f0;border-radius:12px;overflow:hidden;border:1px solid #252a3a;width:100%}
#sk-header{padding:20px 24px 14px;background:linear-gradient(135deg,rgba(99,200,170,.08),rgba(100,130,255,.04));border-bottom:1px solid #252a3a}
#sk-header .ey{font-size:9px;letter-spacing:.2em;color:#63c8aa;text-transform:uppercase;margin-bottom:6px}
#sk-header h2{font-family:'IBM Plex Sans',sans-serif;font-size:18px;font-weight:600;color:#e0e6f0;margin-bottom:10px}
#sk-header .pills{display:flex;gap:8px;flex-wrap:wrap}
.pill{font-size:10px;background:#1a1f2e;border:1px solid #252a3a;padding:3px 10px;border-radius:20px;color:#7a8299}
.pill b{color:#f5a623}
#sk-ctrl{display:flex;align-items:center;gap:10px;padding:12px 20px;background:#13161f;border-bottom:1px solid #252a3a;flex-wrap:wrap}
#sk-ctrl label{font-size:9px;color:#7a8299;letter-spacing:.1em;text-transform:uppercase;white-space:nowrap}
.selw{position:relative;flex:1;min-width:180px;max-width:340px}
.selw::after{content:'\25be';position:absolute;right:9px;top:50%;transform:translateY(-50%);color:#63c8aa;pointer-events:none;font-size:11px}
#sk-ctrl select{width:100%;background:#0f1117;border:1px solid #252a3a;color:#e0e6f0;padding:7px 28px 7px 9px;font-family:'IBM Plex Mono',monospace;font-size:11px;border-radius:6px;cursor:pointer;appearance:none;outline:none;transition:border-color .2s}
#sk-ctrl select:focus{border-color:#63c8aa}
#sk-badge{font-size:10px;color:#7a8299;margin-left:auto}
#sk-badge b{color:#f5a623}
.vtog{display:flex;gap:2px;background:#0f1117;border:1px solid #252a3a;border-radius:6px;padding:2px}
.vbtn{background:transparent;border:none;color:#7a8299;padding:5px 11px;font-family:'IBM Plex Mono',monospace;font-size:10px;cursor:pointer;border-radius:4px;transition:all .15s}
.vbtn.on{background:#1a1f2e;color:#e0e6f0}
/* ── Split view ── */
#sk-body{display:grid;grid-template-columns:250px 1fr}
#sk-sb{border-right:1px solid #252a3a;overflow-y:auto;max-height:520px}
#sk-sb-hd{padding:8px 13px;font-size:8px;letter-spacing:.2em;text-transform:uppercase;color:#7a8299;border-bottom:1px solid #252a3a;position:sticky;top:0;background:#0f1117;z-index:5}
.rr{display:flex;align-items:center;gap:7px;padding:8px 13px;cursor:pointer;border-bottom:1px solid rgba(37,42,58,.5);transition:background .15s}
.rr:hover{background:#13161f}
.rr.ac{background:rgba(99,200,170,.06);border-left:2px solid #63c8aa}
.rr.fd{opacity:.25}
.rnum{font-size:8px;color:#7a8299;width:18px;text-align:right;flex-shrink:0}
.rdot{width:7px;height:7px;border-radius:50%;background:#63c8aa;flex-shrink:0;transition:transform .15s}
.rr.ac .rdot{transform:scale(1.4);box-shadow:0 0 6px #63c8aa}
.rname{font-size:10px;color:#e0e6f0;flex:1;line-height:1.3}
.rmb{font-size:8px;color:#7a8299;background:#1a1f2e;padding:2px 5px;border-radius:3px;flex-shrink:0}
.rrep{font-size:8px;color:#63c8aa;background:rgba(99,200,170,.1);border:1px solid rgba(99,200,170,.25);padding:2px 5px;border-radius:3px;flex-shrink:0}
/* ── Detail panel ── */
#sk-det{padding:20px 24px;overflow-y:auto;max-height:520px}
.dh{display:flex;align-items:flex-start;justify-content:space-between;gap:14px;margin-bottom:18px;flex-wrap:wrap}
.dey{font-size:8px;letter-spacing:.2em;text-transform:uppercase;color:#63c8aa;margin-bottom:4px}
.dt{font-family:'IBM Plex Sans',sans-serif;font-size:18px;font-weight:600;color:#e0e6f0;text-transform:capitalize;line-height:1.2}
.scs{display:flex;gap:7px;flex-shrink:0}
.sc{background:#1a1f2e;border:1px solid #252a3a;border-radius:7px;padding:7px 11px;text-align:center;min-width:56px}
.sc-l{font-size:7px;letter-spacing:.12em;text-transform:uppercase;color:#7a8299;margin-bottom:3px}
.sc-v{font-size:19px;font-weight:500;line-height:1}
.sc.SL .sc-v{color:#63c8aa}.sc.SP .sc-v{color:#f5a623}.sc.SU .sc-v{color:#e87070}
.brs{margin-bottom:18px}
.blbl{font-size:8px;color:#7a8299;margin-bottom:8px;letter-spacing:.15em;text-transform:uppercase;padding-bottom:5px;border-bottom:1px solid #252a3a}
.br{display:flex;align-items:center;gap:9px;margin-bottom:7px}
.bn{font-size:9px;color:#7a8299;width:65px;flex-shrink:0}
.bt{flex:1;height:4px;background:#252a3a;border-radius:2px;overflow:hidden}
.bf{height:100%;border-radius:2px;transition:width .5s cubic-bezier(.4,0,.2,1)}
.bf.BL{background:#63c8aa}.bf.BP{background:#f5a623}.bf.BU{background:#e87070}
.bv{font-size:9px;color:#e0e6f0;width:14px;text-align:right}
/* ── Rep cards ── */
.rlbl{font-size:8px;color:#63c8aa;margin-bottom:10px;letter-spacing:.15em;text-transform:uppercase;padding-bottom:5px;border-bottom:1px solid #252a3a}
.rep-cards{display:flex;flex-direction:column;gap:8px;margin-bottom:20px}
.rep-card{background:#13161f;border:1px solid rgba(99,200,170,.2);border-radius:7px;padding:10px 13px}
.rep-card-top{display:flex;align-items:center;justify-content:space-between;gap:8px;margin-bottom:6px}
.rep-card-name{font-size:11px;color:#e0e6f0;text-transform:capitalize;font-weight:500;flex:1}
.rep-card-freq{font-size:9px;color:#f5a623;background:rgba(245,166,35,.1);border:1px solid rgba(245,166,35,.2);padding:1px 7px;border-radius:3px;flex-shrink:0}
.rep-card-primary{font-size:8px;color:#63c8aa;background:rgba(99,200,170,.08);border:1px solid rgba(99,200,170,.2);padding:1px 7px;border-radius:3px;flex-shrink:0}
.rep-card-scores{display:flex;gap:10px}
.rep-cs{display:flex;align-items:center;gap:4px;font-size:9px;color:#7a8299}
.rep-cd{width:5px;height:5px;border-radius:50%}
.rep-cd.L{background:#63c8aa}.rep-cd.P{background:#f5a623}.rep-cd.U{background:#e87070}
/* ── Members ── */
.mlbl{font-size:8px;color:#7a8299;margin-bottom:10px;letter-spacing:.15em;text-transform:uppercase;padding-bottom:5px;border-bottom:1px solid #252a3a}
.mwrap{display:flex;flex-wrap:wrap;gap:6px}
.mt{background:#13161f;border:1px solid #252a3a;border-radius:5px;padding:5px 10px;font-size:10px;color:#7b9cff;text-transform:capitalize;cursor:default}
.mt:hover{border-color:#7b9cff}
/* ── Bubble view ── */
#bub-canvas-wrap{position:relative;width:100%;overflow:hidden}
.bub-tip{position:absolute;background:#1a1f2e;border:1px solid #252a3a;border-radius:6px;padding:8px 12px;font-size:10px;pointer-events:none;z-index:20;max-width:220px;display:none;box-shadow:0 4px 16px rgba(0,0,0,.5)}
.bub-tip-title{font-family:'IBM Plex Sans',sans-serif;font-weight:600;color:#e0e6f0;margin-bottom:4px;text-transform:capitalize;font-size:11px}
.bub-tip-row{display:flex;justify-content:space-between;gap:10px;color:#7a8299;font-size:9px;margin-top:2px}
.bub-tip-row span{color:#e0e6f0}
/* ── Grid view ── */
#sk-grid{display:none;padding:16px 20px}
.gi{display:grid;grid-template-columns:repeat(auto-fill,minmax(210px,1fr));gap:10px}
.gc{background:#13161f;border:1px solid #252a3a;border-radius:8px;padding:12px;cursor:pointer;transition:all .2s;position:relative;overflow:hidden}
.gc::before{content:'';position:absolute;top:0;left:0;right:0;height:2px;background:linear-gradient(90deg,#63c8aa,#7b9cff);opacity:0;transition:opacity .2s}
.gc:hover{border-color:rgba(99,200,170,.35);transform:translateY(-1px)}
.gc:hover::before,.gc.hl::before{opacity:1}
.gc.hl{border-color:#63c8aa;box-shadow:0 0 0 1px #63c8aa}
.gc.dm{opacity:.25;pointer-events:none}
.gct{display:flex;justify-content:space-between;margin-bottom:7px}
.gci{font-size:8px;color:#7a8299;letter-spacing:.1em}
.gsz{font-size:8px;background:#0f1117;color:#7a8299;padding:2px 5px;border-radius:3px}
.gc-reps{margin-bottom:7px}
.gc-rep-primary{font-family:'IBM Plex Sans',sans-serif;font-size:11px;font-weight:600;color:#e0e6f0;text-transform:capitalize;line-height:1.3;margin-bottom:3px}
.gc-rep-extra{font-size:9px;color:#63c8aa;text-transform:capitalize;line-height:1.5}
.gcm{display:flex;flex-wrap:wrap;gap:3px;margin-bottom:7px}
.gmt{font-size:8px;color:#7a8299;background:#0f1117;border:1px solid #252a3a;padding:2px 5px;border-radius:3px;text-transform:capitalize}
.gmx{font-size:8px;color:#7a8299;font-style:italic;padding:2px 4px}
.gsc{display:flex;gap:7px}
.gss{display:flex;align-items:center;gap:3px;font-size:9px}
.gsd{width:5px;height:5px;border-radius:50%}
.gsd.L{background:#63c8aa}.gsd.P{background:#f5a623}.gsd.U{background:#e87070}
::-webkit-scrollbar{width:3px;height:3px}
::-webkit-scrollbar-track{background:#0f1117}
::-webkit-scrollbar-thumb{background:#252a3a;border-radius:2px}
</style>
'''

BODY = '''
<div id="sk-root">
  <div id="sk-header">
    <div class="ey">Stakeholder Analysis &middot; AI-Powered Student Assessment</div>
    <h2>Cluster Map</h2>
    <div class="pills">
      <div class="pill">Clusters <b id="h-nc"></b></div>
      <div class="pill">Stakeholders <b id="h-ns"></b></div>
      <div class="pill">Representatives <b id="h-nr"></b></div>
    </div>
  </div>
  <div id="sk-ctrl">
    <label>Cluster</label>
    <div class="selw"><select id="sk-sel"><option value="all">&#8212; Show all clusters &#8212;</option></select></div>
    <div class="vtog">
      <button class="vbtn on" id="btn-split" onclick="skV('split')">Split</button>
      <button class="vbtn" id="btn-grid" onclick="skV('grid')">Grid</button>
    </div>
    <div id="sk-badge">Viewing <b id="sk-bt">all clusters</b></div>
  </div>
  <div id="sk-body">
    <div id="sk-sb"><div id="sk-sb-hd">All Clusters</div><div id="sk-sbl"></div></div>
    <div id="sk-det"></div>
  </div>
  <div id="sk-grid" style="display:none"><div class="gi" id="sk-gi"></div></div>
  <div class="bub-tip" id="bub-tip">
    <div class="bub-tip-title" id="bt-title"></div>
    <div class="bub-tip-row">Legitimacy <span id="bt-l"></span></div>
    <div class="bub-tip-row">Power      <span id="bt-p"></span></div>
    <div class="bub-tip-row">Urgency    <span id="bt-u"></span></div>
    <div class="bub-tip-row">Reps       <span id="bt-r"></span></div>
    <div class="bub-tip-row">Members    <span id="bt-m"></span></div>
  </div>
</div>
'''

JS_T = '''
<script>
(function(){
var D=__DATA__;
var K=Object.keys(D);
var cap=function(s){return s?s[0].toUpperCase()+s.slice(1):s;};
var cur=null;

// ── Header stats ──────────────────────────────────────────────────────────
document.getElementById('h-nc').textContent=K.length;
document.getElementById('h-ns').textContent=K.reduce(function(a,k){return a+D[k].size;},0);
document.getElementById('h-nr').textContent=K.reduce(function(a,k){return a+D[k].reps.length;},0);
document.getElementById('sk-bt').textContent='all '+K.length+' clusters';

var sel=document.getElementById('sk-sel');
var sbl=document.getElementById('sk-sbl');
var det=document.getElementById('sk-det');
var gi =document.getElementById('sk-gi');
var tip=document.getElementById('bub-tip');

// Dropdown uses primary rep name
K.forEach(function(id){
  var o=document.createElement('option');
  o.value=id; o.textContent='Cluster '+id+' - '+cap(D[id].primary);
  sel.appendChild(o);
});

// ── Urgency → color ───────────────────────────────────────────────────────
function urgColor(u){
  if(u>=5) return '#e87070';
  if(u>=4) return '#f5a623';
  if(u>=3) return '#7b9cff';
  return '#63c8aa';
}

// ── Bubble chart ──────────────────────────────────────────────────────────
function buildBubble(){
  var W=det.clientWidth||600; if(W<300) W=600;
  var PAD=60, H=420;
  var plotW=W-PAD*2, plotH=H-PAD*2;

  function xPos(p){ return PAD+(p-1)/(5-1)*plotW; }
  function yPos(l){ return PAD+(1-(l-2)/(5-2))*plotH; }

  var sizes=K.map(function(k){return D[k].size;});
  var sMax=Math.max.apply(null,sizes), sMin=Math.min.apply(null,sizes);
  function radius(s){ return 10+(Math.sqrt(s)-Math.sqrt(sMin))/(Math.sqrt(sMax)-Math.sqrt(sMin)+1e-9)*26; }

  var gridLines='';
  [1,2,3,4,5].forEach(function(v){
    var x=xPos(v);
    gridLines+='<line x1="'+x+'" y1="'+PAD+'" x2="'+x+'" y2="'+(PAD+plotH)+'" stroke="#252a3a" stroke-width="1" stroke-dasharray="4,4"/>';
    if(v>=2) gridLines+='<line x1="'+PAD+'" y1="'+yPos(v)+'" x2="'+(PAD+plotW)+'" y2="'+yPos(v)+'" stroke="#252a3a" stroke-width="1" stroke-dasharray="4,4"/>';
  });

  var xLabels=''; [1,2,3,4,5].forEach(function(v){ xLabels+='<text x="'+xPos(v)+'" y="'+(H-PAD+18)+'" text-anchor="middle" fill="#7a8299" font-size="9">'+v+'</text>'; });
  var yLabels=''; [2,3,4,5].forEach(function(v){ yLabels+='<text x="'+(PAD-8)+'" y="'+(yPos(v)+4)+'" text-anchor="end" fill="#7a8299" font-size="9">'+v+'</text>'; });
  var axTitles='<text x="'+(PAD+plotW/2)+'" y="'+(H-PAD+34)+'" text-anchor="middle" fill="#7a8299" font-size="9" letter-spacing="1" font-family="IBM Plex Mono,monospace">POWER</text>';
  axTitles+='<text x="'+(PAD-36)+'" y="'+(PAD+plotH/2)+'" text-anchor="middle" fill="#7a8299" font-size="9" letter-spacing="1" font-family="IBM Plex Mono,monospace" transform="rotate(-90,'+(PAD-36)+','+(PAD+plotH/2)+')">LEGITIMACY</text>';

  var bubbles='';
  K.forEach(function(id){
    var c=D[id];
    var x=xPos(c.power), y=yPos(c.legitimacy);
    var r=radius(c.size);
    var col=urgColor(c.urgency);
    // Dashed outer ring signals multi-rep cluster
    if(c.reps.length>1){
      bubbles+='<circle cx="'+x+'" cy="'+y+'" r="'+(r+5)+'" fill="none" stroke="'+col+'" stroke-width="1" stroke-dasharray="3,3" opacity="0.45"/>';
    }
    bubbles+='<circle class="bub" data-id="'+id+'" cx="'+x+'" cy="'+y+'" r="'+r+'" fill="'+col+'" fill-opacity="0.18" stroke="'+col+'" stroke-width="1.5" style="cursor:pointer;transition:fill-opacity .2s"/>';
    var shortName=cap(c.primary);
    if(shortName.length>18) shortName=shortName.substring(0,16)+'…';
    if(r>14){
      bubbles+='<text x="'+x+'" y="'+(y+3)+'" text-anchor="middle" fill="'+col+'" font-size="8" font-family="IBM Plex Mono,monospace" style="pointer-events:none;font-weight:500">'+shortName+'</text>';
    }
  });

  var legend='<div style="display:flex;gap:16px;align-items:center;flex-wrap:wrap;padding:10px 20px;border-top:1px solid #252a3a;background:#13161f">'+
    '<span style="font-size:8px;letter-spacing:.15em;text-transform:uppercase;color:#7a8299">Urgency:</span>'+
    ['#63c8aa:1-2','#7b9cff:3','#f5a623:4','#e87070:5'].map(function(x){
      var p=x.split(':');
      return '<span style="display:flex;align-items:center;gap:4px;font-size:9px;color:#7a8299"><span style="width:8px;height:8px;border-radius:50%;background:'+p[0]+';display:inline-block"></span>'+p[1]+'</span>';
    }).join('')+
    '<span style="display:flex;align-items:center;gap:5px;font-size:9px;color:#7a8299;margin-left:6px">'+
      '<svg width="14" height="14" style="flex-shrink:0"><circle cx="7" cy="7" r="5" fill="none" stroke="#aaa" stroke-width="1" stroke-dasharray="2,2"/></svg>'+
      'multi-rep'+
    '</span>'+
    '<span style="margin-left:auto;font-size:9px;color:#7a8299">size = cluster size &nbsp;|&nbsp; axes = Power vs Legitimacy</span>'+
  '</div>';

  det.innerHTML='<div id="bub-canvas-wrap" style="position:relative;width:100%;overflow:hidden">'+
    '<svg viewBox="0 0 '+W+' '+H+'" width="'+W+'" height="'+H+'" style="display:block;width:100%">'+
      '<rect width="'+W+'" height="'+H+'" fill="#0f1117"/>'+
      gridLines+xLabels+yLabels+axTitles+bubbles+
    '</svg></div>'+legend;

  det.querySelectorAll('.bub').forEach(function(el){
    el.addEventListener('mouseenter',function(){
      var c=D[el.getAttribute('data-id')];
      document.getElementById('bt-title').textContent=cap(c.primary);
      document.getElementById('bt-l').textContent=c.legitimacy;
      document.getElementById('bt-p').textContent=c.power;
      document.getElementById('bt-u').textContent=c.urgency;
      document.getElementById('bt-r').textContent=c.reps.length+' rep'+(c.reps.length!==1?'s':'');
      document.getElementById('bt-m').textContent=c.members.length+' members';
      tip.style.display='block';
      el.setAttribute('fill-opacity','0.45');
      el.setAttribute('stroke-width','2.5');
    });
    el.addEventListener('mousemove',function(e){
      var wrap=document.getElementById('bub-canvas-wrap');
      var rect=wrap.getBoundingClientRect();
      var tx=e.clientX-rect.left+12, ty=e.clientY-rect.top+12;
      if(tx+230>rect.width) tx=e.clientX-rect.left-230;
      tip.style.left=tx+'px'; tip.style.top=ty+'px';
    });
    el.addEventListener('mouseleave',function(){
      tip.style.display='none';
      el.setAttribute('fill-opacity','0.18');
      el.setAttribute('stroke-width','1.5');
    });
    el.addEventListener('click',function(){ sk(el.getAttribute('data-id')); });
  });
}

// ── Sidebar ───────────────────────────────────────────────────────────────
function bSb(){
  sbl.innerHTML='';
  K.forEach(function(id){
    var c=D[id], d=document.createElement('div');
    d.className='rr'; d.dataset.c=id;
    var repBadge=c.reps.length>1?'<div class="rrep">'+c.reps.length+'★</div>':'';
    d.innerHTML='<span class="rnum">'+id+'</span><div class="rdot"></div>'+
      '<div class="rname">'+cap(c.primary)+'</div>'+
      repBadge+
      '<div class="rmb">'+c.members.length+'</div>';
    d.onclick=function(){sk(id);};
    sbl.appendChild(d);
  });
}

// ── Grid ──────────────────────────────────────────────────────────────────
function bGr(){
  gi.innerHTML='';
  K.forEach(function(id){
    var c=D[id], d=document.createElement('div');
    d.className='gc'; d.dataset.c=id;
    // Primary rep always shown; extra reps listed below
    var extraReps=c.reps.slice(1).map(function(r){return cap(r.name);}).join(', ');
    var repsHtml='<div class="gc-reps">'+
      '<div class="gc-rep-primary">'+cap(c.primary)+'</div>'+
      (extraReps?'<div class="gc-rep-extra">+'+extraReps+'</div>':'')+
    '</div>';
    var p=c.members.slice(0,2), ex=c.members.length-p.length;
    d.innerHTML=
      '<div class="gct"><span class="gci">CLUSTER '+id+'</span><span class="gsz">'+c.size+' total</span></div>'+
      repsHtml+
      '<div class="gcm">'+p.map(function(m){return '<span class="gmt">'+cap(m)+'</span>';}).join('')+(ex>0?'<span class="gmx">+'+ex+'</span>':'')+'</div>'+
      '<div class="gsc"><div class="gss"><div class="gsd L"></div>L:'+c.legitimacy+'</div><div class="gss"><div class="gsd P"></div>P:'+c.power+'</div><div class="gss"><div class="gsd U"></div>U:'+c.urgency+'</div></div>';
    d.onclick=function(){skV('split'); sk(id);};
    gi.appendChild(d);
  });
}

// ── Detail panel ──────────────────────────────────────────────────────────
function rDet(id){
  if(!id){ buildBubble(); return; }
  var c=D[id];

  // Score bars use primary rep scores
  var bHtml=[['Legitimacy','BL',c.legitimacy],['Power','BP',c.power],['Urgency','BU',c.urgency]].map(function(b){
    return '<div class="br"><span class="bn">'+b[0]+'</span><div class="bt"><div class="bf '+b[1]+'" style="width:'+b[2]*20+'%"></div></div><span class="bv">'+b[2]+'</span></div>';
  }).join('');

  // One card per representative
  var repsHtml=c.reps.map(function(r){
    var isPrimary=(r.name===c.primary);
    return '<div class="rep-card">'+
      '<div class="rep-card-top">'+
        '<span class="rep-card-name">'+cap(r.name)+'</span>'+
        (isPrimary?'<span class="rep-card-primary">primary</span>':'')+
        '<span class="rep-card-freq">'+r.frequency+' mentions</span>'+
      '</div>'+
      '<div class="rep-card-scores">'+
        '<span class="rep-cs"><span class="rep-cd L"></span>L:'+r.legitimacy+'</span>'+
        '<span class="rep-cs"><span class="rep-cd P"></span>P:'+r.power+'</span>'+
        '<span class="rep-cs"><span class="rep-cd U"></span>U:'+r.urgency+'</span>'+
      '</div>'+
    '</div>';
  }).join('');

  var mHtml=c.members.length>0
    ? c.members.map(function(m){return '<div class="mt">'+cap(m)+'</div>';}).join('')
    : '<p style="color:#7a8299;font-size:11px">No other members.</p>';

  det.innerHTML=
    '<div class="dh">'+
      '<div>'+
        '<div class="dey">Cluster '+id+'</div>'+
        '<div class="dt">'+cap(c.primary)+'</div>'+
      '</div>'+
      '<div class="scs">'+
        '<div class="sc SL"><div class="sc-l">Legit</div><div class="sc-v">'+c.legitimacy+'</div></div>'+
        '<div class="sc SP"><div class="sc-l">Power</div><div class="sc-v">'+c.power+'</div></div>'+
        '<div class="sc SU"><div class="sc-l">Urgency</div><div class="sc-v">'+c.urgency+'</div></div>'+
      '</div>'+
    '</div>'+
    '<div class="brs"><div class="blbl">Primary Score Profile</div>'+bHtml+'</div>'+
    '<div class="rlbl">'+c.reps.length+' Representative'+(c.reps.length!==1?'s':'')+'</div>'+
    '<div class="rep-cards">'+repsHtml+'</div>'+
    '<div class="mlbl">'+c.members.length+' Other Member'+(c.members.length!==1?'s':'')+'</div>'+
    '<div class="mwrap">'+mHtml+'</div>';
}

// ── Select cluster ────────────────────────────────────────────────────────
function sk(id){
  cur=(id==='all')?null:id;
  sel.value=cur||'all';
  document.querySelectorAll('.rr').forEach(function(r){
    r.classList.remove('ac','fd');
    if(cur && r.dataset.c!==String(cur)) r.classList.add('fd');
    if(r.dataset.c===String(cur)){r.classList.add('ac'); r.scrollIntoView({block:'nearest'});}
  });
  document.querySelectorAll('.gc').forEach(function(c){
    c.classList.remove('hl','dm');
    if(cur){(c.dataset.c!==String(cur))?c.classList.add('dm'):c.classList.add('hl');}
  });
  var bt=document.getElementById('sk-bt');
  bt.textContent=cur
    ? 'Cluster '+cur+' · '+D[cur].reps.length+' rep'+(D[cur].reps.length!==1?'s':'')+' · '+D[cur].size+' stakeholders'
    : 'all '+K.length+' clusters';
  rDet(cur);
}

// ── View toggle ───────────────────────────────────────────────────────────
window.skV=function(v){
  document.getElementById('sk-body').style.display=(v==='split')?'grid':'none';
  document.getElementById('sk-grid').style.display=(v==='grid')?'block':'none';
  document.getElementById('btn-split').classList.toggle('on',v==='split');
  document.getElementById('btn-grid').classList.toggle('on',v==='grid');
};
window.sk=sk;
sel.onchange=function(e){sk(e.target.value);};
bSb(); bGr(); sk(null);
})();
</script>
'''

data_json = json.dumps(clusters)
js_final  = JS_T.replace('__DATA__', data_json)
display(HTML(STYLE + BODY + js_final))